In [5]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from datetime import date
from datetime import datetime
import calendar
import os
import re


from scipy import stats
from sklearn.neighbors import KernelDensity
import matplotlib.colors as mcolors
from matplotlib.animation import FuncAnimation, PillowWriter

import seaborn as sns
from pathlib import Path
from tqdm import tqdm_notebook, trange, tqdm
from PIL import Image, ImageDraw
import io
import random as r

from matplotlib.widgets import Slider, Button, RadioButtons
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
from sklearn.pipeline import make_pipeline
# For downsampling
from sklearn.utils import resample,shuffle
from IPython.display import clear_output

In [6]:
df = pd.read_csv("./LotofNADrops.csv")

c:\python38\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [11]:
df.dtypes

Product_Code                                       int16
Primary_Specialty                                 object
State_Comp                                        object
Total_Amount_of_Payment_USDollars                float64
Physician_ID                                      object
Form_of_Payment_or_Transfer_of_Value              object
Product_Category                                  object
Country_Comp                                      object
Third_Party_Recipient                             object
Country_Phys                                      object
City_of_Travel                                    object
Contextual_Information                            object
Product_Name_                                      int16
Country_of_Travel                                 object
Number_of_Payments                                 int32
Related_Product_Indicator                         object
City                                              object
Record_ID                      

In [12]:
df = df.astype(str)
for column in df.columns:
     if(df[column].dtype=='object'):
        df[column] = df[column].str.lower()

#Convert to Date
df['Date']=pd.to_datetime(df['Date'])

#type conv after str conv
df[['Company_ID','Number_of_Payments']]=df[['Company_ID','Number_of_Payments']].astype(int)
df['Total_Amount_of_Payment_USDollars']=df['Total_Amount_of_Payment_USDollars'].astype(float)


columns_encode=['Product_Code','Product_Name_','License_State_','Product_Type','Product_Category','Nature_of_Payment_or_Transfer_of_Value','Form_of_Payment_or_Transfer_of_Value','Zipcode','Third_Party_Recipient', 'Charity', 'Third_Party_Covered','Contextual_Information', 'Related_Product_Indicator','Primary_Specialty']
for column in columns_encode:
    df[column]=df[column].astype('category')
    df[column]=df[column].cat.codes
    
name_list = pd.unique(df[['Country_Comp', 'Country_Phys','Country_of_Travel']].values.ravel('K'))
# name_list = np.delete(name_list,np.where(name_list=="nan"))
name_list_dict = {}
int_code=int(1)
for name in name_list:
    name_list_dict.update({name:int_code})
    int_code+=1

def product_encode(product_name):
    return name_list_dict.get(product_name)

df['Country_Comp'] = df['Country_Comp'].apply(product_encode)
df['Country_Phys'] = df['Country_Phys'].apply(product_encode)
df['Country_of_Travel'] = df['Country_of_Travel'].apply(product_encode)

name_list = pd.unique(df[['State_Comp', 'State_Phys', 'State_of_Travel']].values.ravel('K'))
# name_list = np.delete(name_list,np.where(name_list=="nan"))
name_list_dict = {}
int_code=int(1)
for name in name_list:
    name_list_dict.update({name:int_code})
    int_code+=1

def product_encode(product_name):
    return name_list_dict.get(product_name)

df['State_Comp'] = df['State_Comp'].apply(product_encode)
df['State_Phys'] = df['State_Phys'].apply(product_encode)
df['State_of_Travel'] = df['State_of_Travel'].apply(product_encode)

name_list = pd.unique(df[['City_of_Travel', 'City']].values.ravel('K'))
# name_list = np.delete(name_list,np.where(name_list=="nan"))
name_list_dict = {}
int_code=int(1)
for name in name_list:
    name_list_dict.update({name:int_code})
    int_code+=1

def product_encode(product_name):
    return name_list_dict.get(product_name)

df['City_of_Travel'] = df['City_of_Travel'].apply(product_encode)
df['City'] = df['City'].apply(product_encode)

df['Ownership_Indicator']=np.where((df.Ownership_Indicator == "yes"),1,df.Ownership_Indicator)
df['Ownership_Indicator']=np.where((df.Ownership_Indicator == "no"),0,df.Ownership_Indicator)
df['Ownership_Indicator']=df['Ownership_Indicator'].astype(int)

df['Date_year']=df['Date'].dt.year
df['Date_month']=df['Date'].dt.month
df['Date_day']=df['Date'].dt.day
df.drop(columns=['Date'],inplace=True)

In [32]:
pd.set_option('display.max_columns', None)
df.head()

,Product_Code,Primary_Specialty,State_Comp,Total_Amount_of_Payment_USDollars,Physician_ID,Form_of_Payment_or_Transfer_of_Value,Product_Category,Country_Comp,Third_Party_Recipient,Country_Phys,City_of_Travel,Contextual_Information,Product_Name_,Country_of_Travel,Number_of_Payments,Related_Product_Indicator,City,Record_ID,State_of_Travel,State_Phys,Zipcode,Company_ID,set,Charity,Third_Party_Covered,Nature_of_Payment_or_Transfer_of_Value,Ownership_Indicator,Product_Type,License_State_,Date_year,Date_month,Date_day
0,2723,30,1,48.78,2455,2,798,1,2,1,1,14420,5946,23,1,1,51,1,32,21,1742,597,train,0,0,8,0,2,29,2013,8,1
1,1007,30,2,122.43,1534,2,798,1,2,1,1,14420,8422,23,1,1,58,2,32,13,646,593,train,1,0,8,0,4,29,2013,8,1
2,2723,119,3,10.26,2632,2,798,1,2,1,1,14420,182,23,1,1,219,3,32,26,2804,760,test,0,0,6,0,4,29,2013,8,1
3,2723,119,3,13.68,550,2,798,1,2,1,1,14420,182,23,1,1,1498,4,32,3,5214,760,train,0,0,6,0,4,29,2013,8,1
4,1614,159,2,17.17,1710,2,798,1,2,1,1,14420,3522,23,1,1,26,5,32,15,2536,826,train,0,0,8,0,4,29,2013,8,1


In [13]:
###safekeeping
df_safe = df

###SPLIT
df_test=df[df.set=="test"]
df_train=df[df.set=="train"]

###drop set column
df_test.drop(columns=['set'],inplace=True)
df_train.drop(columns=['set'],inplace=True)

##send predicted attribute to end
first_col = df_train.pop("Ownership_Indicator")
df_train.insert((df_train.shape[1]),"Ownership_Indicator",first_col)

first_col = df_test.pop("Ownership_Indicator")
df_test.insert((df_test.shape[1]),"Ownership_Indicator",first_col)

##send Physician ID to end
first_col = df_train.pop("Physician_ID")
df_train.insert((df_train.shape[1]),"Physician_ID",first_col)

first_col = df_test.pop("Physician_ID")
df_test.insert((df_test.shape[1]),"Physician_ID",first_col)

##send Record ID to end
first_col = df_train.pop("Record_ID")
df_train.insert((df_train.shape[1]),"Record_ID",first_col)

first_col = df_test.pop("Record_ID")
df_test.insert((df_test.shape[1]),"Record_ID",first_col)




c:\python38\lib\site-packages\pandas\core\frame.py:4167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [14]:
np.random.seed(2021)
###downsampling
df_train_maj=df_train[df_train.Ownership_Indicator==0]
df_train_min=df_train[df_train.Ownership_Indicator==1]
df_train_maj_down=resample(df_train_maj, replace=False, n_samples=df_train_min.shape[0], random_state=2021)
df_train_final = pd.concat([df_train_maj_down, df_train_min])

###test train split of own train set
df_train_final['is_train']=np.random.uniform(0,1,len(df_train_final)) <=.80
train, test = df_train_final[df_train_final['is_train']==True], df_train_final[df_train_final['is_train']==False]
train = shuffle(train,random_state=2021)
test = shuffle(test,random_state=2021)

In [20]:
print(df_train_min.shape)
print(train.shape)
train.columns[0:train.shape[1]-4]

(14615, 31)
(23354, 32)


Index(['Product_Code', 'Primary_Specialty', 'State_Comp',
       'Total_Amount_of_Payment_USDollars',
       'Form_of_Payment_or_Transfer_of_Value', 'Product_Category',
       'Country_Comp', 'Third_Party_Recipient', 'Country_Phys',
       'City_of_Travel', 'Contextual_Information', 'Product_Name_',
       'Country_of_Travel', 'Number_of_Payments', 'Related_Product_Indicator',
       'City', 'State_of_Travel', 'State_Phys', 'Zipcode', 'Company_ID',
       'Charity', 'Third_Party_Covered',
       'Nature_of_Payment_or_Transfer_of_Value', 'Product_Type',
       'License_State_', 'Date_year', 'Date_month', 'Date_day'],
      dtype='object')

In [26]:
best=0
i=0
n=150
f = open("AfterSqueezingNEstSearch.txt", "a")
f.write("File Starts here:\n")

while n>0:
    n_est=r.randint(50,200)
    ###feature and label split
    features=train[train.columns[0:(train.shape[1]-4)]]
    y=train['Ownership_Indicator']
    x_test = test[test.columns[0:(test.shape[1]-4)]]
    y_test = test['Ownership_Indicator']

    ###model training
    clf = RandomForestClassifier(n_jobs=-1, oob_score = True, n_estimators = n_est, random_state=2021)
    clf.fit(features, y)
    test_acc=clf.score(x_test,y_test)
    if test_acc>best:
        best=test_acc
        f.write('BEST:'+' '+str(test_acc)+' N_EST: '+str(n_est)+'\n')
        print('BEST:'+' '+str(test_acc)+' N_EST: '+str(n_est)+'\n')
    score_test=clf.predict(x_test)
    if test_acc>0.9991490810074881:
        f.close()
        break

BEST: 0.9988087134104833 N_EST: 189

BEST: 0.9989788972089857 N_EST: 57

BEST: 0.9991490810074881 N_EST: 70



KeyboardInterrupt: 

In [28]:

###feature and label split
features=train[train.columns[0:(train.shape[1]-4)]]
y=train['Ownership_Indicator']
x_test = test[test.columns[0:(test.shape[1]-4)]]
y_test = test['Ownership_Indicator']

###model training
clf = RandomForestClassifier(n_jobs=-1, oob_score = True, n_estimators = 70, random_state=2021)
clf.fit(features, y)
test_acc=clf.score(x_test,y_test)

    # Accuracy Scores
print ('Internal Accuracy Score', clf.oob_score_)
print ('RF accuracy: TRAINING', clf.score(features,y))
print ('RF accuracy: TESTING', test_acc)
score_test=clf.predict(x_test)

Internal Accuracy Score 0.9990151580029117
RF accuracy: TRAINING 1.0
RF accuracy: TESTING 0.9991490810074881


In [29]:
###Creating Dataframes to find accuracy of Physician Level

finalAccPhysID=test['Physician_ID']
finalAcc={'Physician_ID':finalAccPhysID, 'Score':score_test}
finalAccDF=pd.DataFrame(data=finalAcc)
# finalAccDF['Physician_ID'].unique().size
physUFAD=finalAccDF['Physician_ID'].unique()
sc=[None]*physUFAD.size
for i in range(physUFAD.size):
    physician=physUFAD[i]
    if(finalAccDF.query('Physician_ID==@physician and Score==1').shape[0]>0):
        sc[i]=1
    else:
        sc[i]=0
        
finTestPredict={'Physician_ID':physUFAD, 'Score':sc}
finTestPredictDF=pd.DataFrame(data=finTestPredict)

TrueFinalAcc={'Physician_ID':finalAccPhysID, 'Score':test['Ownership_Indicator']}
TrueFinalAccDF=pd.DataFrame(data=TrueFinalAcc)
# finalAccDF['Physician_ID'].unique().size
physUFAD=TrueFinalAccDF['Physician_ID'].unique()
sc=[None]*physUFAD.size
for i in range(physUFAD.size):
    physician=physUFAD[i]
    if(TrueFinalAccDF.query('Physician_ID==@physician and Score==1').shape[0]>0):
        sc[i]=1
    else:
        sc[i]=0

TrueFinTest={'Physician_ID':physUFAD, 'Score':sc}
TrueFinTestDF=pd.DataFrame(data=TrueFinTest)

In [30]:
ownTestScore=TrueFinTestDF['Score']-finTestPredictDF['Score']
print(ownTestScore)
accuracy=(ownTestScore.value_counts(dropna=False,sort=False,ascending=True).loc[0])/ownTestScore.size*100
print(accuracy)

0       0
1       0
2       0
3       0
4       0
       ..
1777    0
1778    0
1779    0
1780    0
1781    0
Name: Score, Length: 1782, dtype: int64
99.83164983164983


In [38]:
phys_id_list=df_safe[df_safe.set=="test"].Physician_ID

In [34]:
predict_set=df_test
predict_set.drop(columns=['Physician_ID','Ownership_Indicator','Record_ID'],inplace=True)
predict_set.columns

c:\python38\lib\site-packages\pandas\core\frame.py:4167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


Index(['Product_Code', 'Primary_Specialty', 'State_Comp',
       'Total_Amount_of_Payment_USDollars',
       'Form_of_Payment_or_Transfer_of_Value', 'Product_Category',
       'Country_Comp', 'Third_Party_Recipient', 'Country_Phys',
       'City_of_Travel', 'Contextual_Information', 'Product_Name_',
       'Country_of_Travel', 'Number_of_Payments', 'Related_Product_Indicator',
       'City', 'State_of_Travel', 'State_Phys', 'Zipcode', 'Company_ID',
       'Charity', 'Third_Party_Covered',
       'Nature_of_Payment_or_Transfer_of_Value', 'Product_Type',
       'License_State_', 'Date_year', 'Date_month', 'Date_day'],
      dtype='object')

In [35]:
score = clf.predict(predict_set)

In [39]:
submit={'Physician_ID':phys_id_list, 'Score':score}
submit_df=pd.DataFrame(data=submit)
physUnique=submit_df['Physician_ID'].unique()

In [41]:

sc=[None]*1000
for i in range(physUnique.size):
    physician=physUnique[i]
    if(submit_df.query('Physician_ID==@physician and Score==1').shape[0]>0):
        sc[i]=1
    else:
        sc[i]=0

In [42]:
submit_df['Physician_ID'].unique().size

1000

In [43]:
final_submit={'id':physUnique, 'prediction':sc}
fs_df=pd.DataFrame(data=final_submit)

In [45]:
inspect1 = pd.read_csv("./submission2.csv")
for i in range(1000):
    ids=fs_df.iloc[i].id
    if(inspect1.query('id==@ids').shape[0]==0):
        print("Shits WRONG")

In [46]:
fs_df.to_csv("./submission5test.csv",index=False)